In [ ]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from  tqdm import tqdm_notebook
import tensorflow as tf
import time
from os.path import join

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


In [ ]:
npix = 28

fig = plt.figure(figsize=(12, 4))
P = np.random.permutation(10000)
for i in range(10):
    ax = fig.add_subplot(1, 10, i+1)
    idx = P[i]
    ax.matshow(np.reshape(mnist.train.images[idx, :], (npix, npix)), cmap=plt.cm.Greys_r)
    ax.set_title(str(np.argmax(mnist.train.labels[idx])))
    ax.axis('off')
               

In [ ]:
tf.reset_default_graph()

tf_input = tf.placeholder(tf.float32, [None, 784])
#z = tf.layers.conv2d(inputs=tf.reshape(tf_input, [-1, 28, 28, 1]), 
#                     kernel_size=3, strides=1, filters=10, use_bias=True, activation=tf.nn.relu)
#z = tf.layers.max_pooling2d(inputs=z, pool_size=2, strides=2)
#z = tf.layers.flatten(z)
#z = tf.layers.dense(inputs=z, units=10, use_bias=True, activation=tf.nn.relu)
y = tf.layers.dense(inputs=tf_input, units=10, use_bias=True, activation=tf.nn.softmax)
tf_label = tf.placeholder(tf.float32, [None, 10])
loss = tf.reduce_mean(-tf.reduce_sum(tf_label * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(learning_rate=1e-2).minimize(loss)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(tf_label,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

nepochs = 20
batch_size = 32
metrics_train = np.zeros(shape=(nepochs, 2))
metrics_test = np.zeros(shape=(nepochs, 2))

config = tf.ConfigProto(device_count = {'GPU': 0}) # USE CPU
with tf.Session(config=config) as sess:
    tf.global_variables_initializer().run()
    
    for epoch in tqdm_notebook(range(nepochs), desc='Train epoch'):
        for _ in range(int(mnist.train.images.shape[0]/batch_size)):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={tf_input: batch_xs, tf_label: batch_ys})
        acc_value, loss_value = sess.run([accuracy, loss], 
                                         feed_dict={tf_input: mnist.train.images, 
                                                    tf_label: mnist.train.labels})
        metrics_train[epoch, 0], metrics_train[epoch, 1] = acc_value, loss_value
        acc_value, loss_value = sess.run([accuracy, loss], 
                                         feed_dict={tf_input: mnist.test.images, 
                                                    tf_label: mnist.test.labels})
        metrics_test[epoch, 0], metrics_test[epoch, 1] = acc_value, loss_value

fig = plt.figure(figsize=(8, 3))
for i, text in enumerate(['Accuraccy', 'Loss']):
    ax = fig.add_subplot(1, 2, i+1)
    ax.set_title(text)
    ax.plot(metrics_train[:, i], linewidth=2, alpha=0.75, label='train')
    ax.plot(metrics_test[:, i], linewidth=2, alpha=0.75, label='test')
    plt.legend(); plt.grid();
plt.tight_layout()